<a href="https://colab.research.google.com/github/openworm/OpenData/blob/master/movement/IntegratingMovementMetadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You will need a github account with your own fork of [PyOpenWorm](https://github.com/openworm/PyOpenWorm) and [OpenWormData](https://github.com/openworm/OpenWormData). Simply press fork in top right hand corner whilst logged into your own github account. Manually add the [movementmetadata.py](https://raw.githubusercontent.com/openworm/OpenData/master/movementmetadata.py) to your forked version of PyOpenWorm. Click "Create New File" file in your forked version of PyOpenWorm. Set the name as "movementmetadata.py" and copy and paste the script. Click "Commit" at the bottom of the page. "movementmetadata.py" stores the classes for categorising and incorporating movement metadata into PyOpenWorm. If you make a mistake and need to delete a repository, go to the home page of your forked version, click Settings and scroll down to Danger Zone. To start again in a google colab environment, click the Runtime tab and the Reset all Runtimes. This will remove the state of all runtimes including local variables and files.** 

**Create ssh keypairs. Leave each of the 3 boxes that will appear empty by pressing Enter.**

In [0]:
!ssh-keygen

**View the public ssh key that you have generated. You can now copy and paste the whole key as a deploy key for your forked version of the OpenWormData repository. Navigate to the homepage of your forked version of OpenWormData through the github website. Click "Settings" and then click "Deploy Keys". It is one of the tabs on the left hand side of the page. Click "Add New Deploy Ke"y and paste the entire key into the empty key box. Make up a title, I designated mine "googlecolab". Tick "Allow Write Access" and press "Add Key".** 

In [0]:
%pycat /root/.ssh/id_rsa.pub

**Get the github ssh host key**

In [0]:
ssh-keyscan github.com >> /root/.ssh/known_hosts

**Clone the PyOpenWorm repository into your Google Colab Working Directory** 

In [0]:
!git clone git@github.com:EST09/PyOpenWorm.git

**Change working directory to PyOpenWorm (the folder you have just cloned from github in the last step)**

In [0]:
%cd PyOpenWorm

**Install PyOpenWorm as has setup.py in its root folder**

In [0]:
!python setup.py install

**pow clone: This will clone a repository into .pow in your current working directory. After a successful clone, a binary database usable as a PyOpenWorm store will have been created from the serialized graphs in the repository.**

In [0]:
!pow clone git@github.com:EST09/OpenWormData.git

**Running the next cell will run tests to check that PyOpenWorm is working as expected**

In [0]:
#!python setup.py test

**Change directory into .pow which we cloned previously**

In [0]:
%cd .pow

**To add an sqlite source we need to modify the pow.conf file. The pow.conf file stores the configuration and settings for the data PyOpenWorm accesses (?). The rdf source is changed to sqlite (why specifically?)**

In [0]:
%%writefile pow.conf 
{
    "data_context_id": "http://openworm.org/data",
    "imports_context_id": "http://openworm.org/data/imports",
    "rdf.source": "sqlite",
    "rdf.store_conf": "worm.db",
    "rdf.upload_block_statement_count": 50
}

In [0]:
%cd ../

**Clone the rdflib-sqlalchemy-1 repository into the PyOpenWorm workng directory**



In [0]:
!git clone git@github.com:mwatts15/rdflib-sqlalchemy-1.git

In [0]:
%cd rdflib-sqlalchemy-1

**Install rdflib-sqlalchemy-1**

In [0]:
!python setup.py install

In [0]:
%cd ../

**Recreate the database from serialized graphs, including uncommited changes**

In [0]:
!pow regendb

**POWmovement_incorporation.py is the script for reading movement metadata and integrating it into PyOpenWorm in the format defined in movementmetadata. wget adds POWmovement_incorporation.py to PyOpenWorm so that it is available as a module for our PyOpenWorm package** 

In [0]:
#where POWmovement_incorporation is the script for uploading the data
!wget -N https://raw.githubusercontent.com/openworm/OpenData/master/POWmovement_incorporation.py

In [0]:
!pip install sickle

**pow save: integrates the module with other modules within within PyOpenWorm allowing data in a given context to be saved in .pow** 

**"(Mark Watts 18:49:
In the example I gave, namepace is a _POWSaveNamespace as defined in PyOpenWorm.command.py
the "pow save" command provides it to your method. You call "pow save" and that calls your method")**

In [0]:
!pow save POWmovement_incorporation

**Commit these files to your forked version of OpenWormData**

In [0]:
!pow commit -m "commit movementmetadata records"

**Push these files to your forked version of OpenWormData**

In [0]:
!pow git push

In [0]:
################################################

In [0]:
I now need to be able to access the movement metadata

In [0]:
%cd examples

In [0]:
%%writefile default.conf
{
    "rdf.source" : "sqlite",
    "rdf.store_conf" : "../.pow/worm.db",
    "rdf.upload_block_statement_count" : 50
}

In [0]:
%cd ../

In [0]:
%cd tests

In [0]:
%%writefile data_integrity_test.conf
{
    "connectomecsv" : "OpenWormData/aux_data/connectome.csv",
    "neuronscsv" : "OpenWormData/aux_data/neurons.csv",
    "rdf.source" : "sqlite",
    "rdf.store_conf" : ".pow/worm.db",
    "rdf.upload_block_statement_count" : 50
}

In [0]:
%cd ../

In [0]:
%%writefile pow-test.py
import PyOpenWorm as P
conn = P.connect(".pow/pow.conf")
from PyOpenWorm.context import Context
ctx = Context(ident='http://openworm.org/entities/bio#worm0-data', conf=conn.conf)

# Grabs the representation of the neuronal network
from PyOpenWorm.worm import Worm
net = ctx.stored(Worm)().neuron_network()

# Grab a specific neuron
from PyOpenWorm.neuron import Neuron
aval = ctx.stored(Neuron)(name='AVAL')

print(aval.type.one())
#'interneuron'

#show how many connections go out of AVAL
print(aval.connection.count('pre'))

print(aval.name())


#list all known receptors
print(sorted(aval.receptors()))
#['GGR-3', 'GLR-1', ... 'NPR-4', 'UNC-8']

#show how many chemical synapses go in and out of AVAL
print(aval.connection.count('either', syntype='send'))
#105

muscles = ctx.stored(Worm)().muscles()
print(len(muscles))

In [0]:
!python pow-test.py

interneuron
86
AVAL
['GGR-3', 'GLR-1', 'GLR-2', 'GLR-4', 'GLR-5', 'NMR-1', 'NMR-2', 'NPR-4', 'UNC-8']
105
158


In [0]:
%%writefile pow-test.py
import PyOpenWorm as P
conn = P.connect(".pow/pow.conf")
#conn = P.connect("readme.conf")
from PyOpenWorm.worm import Worm
from movementmetadata import MovementMetadata
from PyOpenWorm.context import Context


#ctx = Context(ident='http://openworm.org/entities/bio#worm0-data', conf=conn.conf)
ctx = Context(ident='http://openworm.org/data/movement/10.5281/zenodo.1014773')
#ctx = Context(ident='http://openworm.org/data/movement', conf=conn.conf)


print(ctx)
from PyOpenWorm.neuron import Neuron
aval = ctx.stored(Neuron)(name='AVAL')
print(aval.type.one())

#net = ctx.stored(Worm)()
record = ctx.stored(MovementMetadata)(key = "10.5281/zenodo.1014773")
print(record)
col = record.collection
print(col.__dict__)
print(col.__dict__.keys())
#print(record.collection.collector())





In [0]:
!python pow-test.py

PyOpenWorm.context.Context(ident="http://openworm.org/data/movement/10.5281/zenodo.1014773")
None
MovementMetadata(http://openworm.org/entities/MovementMetadata/10.5281/zenodo.1014773)
{'_Contextualizable__context': PyOpenWorm.context.QueryContext(ident="http://openworm.org/data/movement/10.5281/zenodo.1014773"), '_contexts': <WeakValueDictionary at 0x7f1dbffc4cf8>, '_Configureable__conf': <PyOpenWorm.configure.ImmutableConfigure object at 0x7f1dc214eac8>, '_DataUser__base_namespace': None, '_v': [], 'owner': MovementMetadata(ident=rdflib.term.URIRef('http://openworm.org/entities/MovementMetadata/10.5281/zenodo.1014773')), '_hdf': {}, 'filling': False}
dict_keys(['_Contextualizable__context', '_contexts', '_Configureable__conf', '_DataUser__base_namespace', '_v', 'owner', '_hdf', 'filling'])
